# Multi-outputs Task

In [ ]:
!pip install polars[pandas]

In [2]:
import numpy
import pandas # or use `polars`
import torch

## Data Processing from scikit-learn

In [3]:
from sklearn.datasets import make_multilabel_classification

In [4]:
X, y = make_multilabel_classification(n_samples=1000, 
                                      n_features=10,
                                      n_classes=3,
                                      n_labels=2,
                                      random_state=1)

In [5]:
X.shape, y.shape

((1000, 10), (1000, 3))

## Machine Learning Process

### Load Perming and Config Hyperparameters


In [6]:
import perming
main = perming.Box(10, 3, (30,), batch_size=8, activation='relu', inplace_on=True, solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.Ranker(10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Multi-outputs'](10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=10, out_features=30, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=30, out_features=3, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MultiLabelSoftMarginLoss()),
             ('batch_size', 8),
             ('solver',
              SGD (
              Parameter Group 0
                  dampening: 0
                  lr: 0.01
                  momentum: 0
                  nesterov: False
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### Dataloader from Numpy with Multi-threaded


In [7]:
main.data_loader(X, y, random_seed=0)

### Training Stage and Accelerated Validation


In [8]:
main.train_val(num_epochs=40, interval=10, tolerance=1e-4, patience=10, early_stop=True)

Epoch [1/40], Step [10/100], Training Loss: 0.4588, Validation Loss: 0.6206
Epoch [1/40], Step [20/100], Training Loss: 0.4977, Validation Loss: 0.5936
Epoch [1/40], Step [30/100], Training Loss: 0.6046, Validation Loss: 0.5684
Epoch [1/40], Step [40/100], Training Loss: 0.5572, Validation Loss: 0.5566
Epoch [1/40], Step [50/100], Training Loss: 0.6522, Validation Loss: 0.5514
Epoch [1/40], Step [60/100], Training Loss: 0.3386, Validation Loss: 0.5348
Epoch [1/40], Step [70/100], Training Loss: 0.5743, Validation Loss: 0.5256
Epoch [1/40], Step [80/100], Training Loss: 0.3826, Validation Loss: 0.5162
Epoch [1/40], Step [90/100], Training Loss: 0.5820, Validation Loss: 0.5080
Epoch [1/40], Step [100/100], Training Loss: 0.5111, Validation Loss: 0.5049
Epoch [2/40], Step [10/100], Training Loss: 0.2895, Validation Loss: 0.4991
Epoch [2/40], Step [20/100], Training Loss: 0.3002, Validation Loss: 0.4889
Epoch [2/40], Step [30/100], Training Loss: 0.4480, Validation Loss: 0.4856
Epoch [2/40

### Test Model with Loss


In [9]:
main.test()

loss of Box on the 104 test dataset: 0.20897558331489563. accuracy: 0.0000 %


OrderedDict([('problem', 'multi-outputs'),
             ('loss',
              {'train': 0.21326859295368195,
               'val': 0.2859066426753998,
               'test': 0.20897558331489563})])

In [10]:
X, y = torch.as_tensor(X, dtype=torch.float).to(torch.device("cuda")), torch.as_tensor(y, dtype=torch.float).to(torch.device("cuda"))

In [11]:
pred = main.model(X) # predicted

In [12]:
'{}%'.format(100 * sum(row.all().int().item() for row in (pred.ge(0.5) == y)) / X.shape[0])

'72.8%'

### Save Model Parameters to Models Folder


In [13]:
main.save(show=False, dir='../models/outputs.ckpt')

### Load Model Parameters from Models Folder

In [14]:
main.load(show=False, dir='../models/outputs.ckpt')